# Registo no GenBank

## Verificar as anotações correspondentes aos genes de interesse;

In [6]:
from scripts_python import gbSearch
import os
from Bio import SeqIO, Entrez

a = %pwd
wd = (a.rsplit('/',1))[0]

seq_id = "NM_001114380"

In [7]:
handle = Entrez.efetch(db="nucleotide",
                       id=seq_id,
                       rettype="gb",
                       retmode="text")

file_path = os.path.join(wd,"data/reference/ITGAL", "ITGAL_gb.gb")
with open(file_path, 'w') as file:
    file.write(handle.read())

In [8]:
path = os.path.join(wd, "data/reference/ITGAL", "ITGAL_gb.gb")
seq_r = SeqIO.read(path, format="genbank")
for Key, Value in seq_r.annotations.items():
    print(Key,": ", Value)


molecule_type :  mRNA
topology :  linear
data_file_division :  PRI
date :  12-DEC-2020
accessions :  ['NM_001114380']
sequence_version :  2
keywords :  ['RefSeq']
source :  Homo sapiens (human)
organism :  Homo sapiens
taxonomy :  ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo']
references :  [Reference(title='Perlecan regulates pericyte dynamics in the maintenance and repair of the blood-brain barrier', ...), Reference(title='Aggregatibacter actinomycetemcomitans leukotoxin causes activation of lymphocyte function-associated antigen 1', ...), Reference(title='The down-regulation of hsa_circ_0012919, the sponge for miR-125a-3p, contributes to DNA methylation of CD11a and CD70 in CD4(+) T cells of systemic lupus erythematous', ...), Reference(title='Direction of actin flow dictates integrin LFA-1 orientation during leukocyte migration', ...), Reference(

## Verificar e analisar a informação complementar fornecida pela lista de features e seus qualifiers

In [10]:
for feature in seq_r.features:
    if feature.type in ["gene", "CDS"]:
        if feature.qualifiers["gene"][0] == "ITGAL":
            interest_gene = feature
            print(feature.type)
            print(feature.location)
            print(feature.ref_db)
            for Key,Value in feature.qualifiers.items():
                print(Key,": ", Value)
            print(" ")

gene
[0:4877](+)
None
gene :  ['ITGAL']
gene_synonym :  ['CD11A; LFA-1; LFA1A']
note :  ['integrin subunit alpha L']
db_xref :  ['GeneID:3683', 'HGNC:HGNC:6148', 'MIM:153370']
 
CDS
[96:3357](+)
None
gene :  ['ITGAL']
gene_synonym :  ['CD11A; LFA-1; LFA1A']
note :  ['isoform b precursor is encoded by transcript variant 2; LFA-1A; antigen CD11A (p180), lymphocyte function-associated antigen 1, alpha polypeptide; integrin, alpha L (antigen CD11A (p180), lymphocyte function-associated antigen 1; alpha polypeptide); LFA-1 alpha; integrin gene promoter; CD11 antigen-like family member A; leukocyte adhesion glycoprotein LFA-1 alpha chain; leukocyte function-associated molecule 1 alpha chain; integrin alpha-L']
codon_start :  ['1']
product :  ['integrin alpha-L isoform b precursor']
protein_id :  ['NP_001107852.1']
db_xref :  ['CCDS:CCDS45461.1', 'GeneID:3683', 'HGNC:HGNC:6148', 'MIM:153370']
translation :  ['MKDSCITVMAMALLSGFFFFAPASSYNLDVRGARSFSPPRAGRHFGYRVLQVGNGVIVGAPGEGNSTGSLYQCQSGTGHCLPVT

## Referências em outras bases de dados

In [11]:
refs = []

for feature in seq_r.features:
    if "db_xref" in feature.qualifiers:
        refs.extend(feature.qualifiers["db_xref"])

for ref in refs:
    print(ref)

taxon:9606
GeneID:3683
HGNC:HGNC:6148
MIM:153370
CCDS:CCDS45461.1
GeneID:3683
HGNC:HGNC:6148
MIM:153370
